In [1]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

import os
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

# 그래프 시각화
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
baseball_data = pd.read_excel('/Users/jun/Git/407_Project/KBO_prediction_data/baseball_2023.xlsx')

In [3]:
baseball_data.head()

,순,T_ID,팀,G,AB,PA,RUN,안타,2타,3타,...,희타,희비,OOO,OBP,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1,NC,23,25,81,76,14,26,5,1,...,1,1,0.342,0.363,0.513,0.876,0.389,133.7,0.59,0.21
1,2,LG,23,30,94,81,16,27,6,0,...,1,1,0.333,0.409,0.407,0.816,0.386,118.2,0.53,0.24
2,3,KT,23,31,91,82,20,25,6,1,...,0,2,0.305,0.352,0.512,0.864,0.383,115.5,0.45,0.18
3,4,두산,23,24,83,70,12,18,2,0,...,2,1,0.257,0.346,0.371,0.717,0.331,97.7,0.29,0.64
4,5,키움,23,24,85,78,10,24,7,1,...,0,0,0.308,0.365,0.423,0.788,0.364,107.0,0.23,0.90


In [4]:
baseball_data.columns

Index(['순', 'T_ID', '팀', 'G', 'AB', 'PA', 'RUN', '안타', '2타', '3타', '홈런', '루타',
       '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', 'OOO',
       'OBP', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR*', 'WPA'],
      dtype='object')

### 데이터의 결측치와 데이터 형태 확인

In [5]:
baseball_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   순       10 non-null     int64  
 1   T_ID    10 non-null     object 
 2   팀       10 non-null     int64  
 3   G       10 non-null     int64  
 4   AB      10 non-null     int64  
 5   PA      10 non-null     int64  
 6   RUN     10 non-null     int64  
 7   안타      10 non-null     int64  
 8   2타      10 non-null     int64  
 9   3타      10 non-null     int64  
 10  홈런      10 non-null     int64  
 11  루타      10 non-null     int64  
 12  타점      10 non-null     int64  
 13  도루      10 non-null     int64  
 14  도실      10 non-null     int64  
 15  볼넷      10 non-null     int64  
 16  사구      10 non-null     int64  
 17  고4      10 non-null     int64  
 18  삼진      10 non-null     int64  
 19  병살      10 non-null     int64  
 20  희타      10 non-null     int64  
 21  희비      10 non-null     int64  
 22  OOO  

In [6]:
baseball_data.describe()

,순,팀,G,AB,PA,RUN,안타,2타,3타,홈런,...,희타,희비,OOO,OBP,장타,OPS,wOBA,wRC+,WAR*,WPA
count,10.00000,10.0,10.00000,10.000000,10.00000,10.000000,10.000000,10.00000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,5.50000,23.0,25.40000,83.800000,73.50000,11.900000,20.800000,3.90000,0.500000,1.300000,...,0.900000,0.700000,0.280900,0.355300,0.399600,0.755000,0.349600,100.480000,0.277000,0.299000
std,3.02765,0.0,2.75681,6.646637,7.33712,3.842742,4.516636,2.13177,0.527046,1.159502,...,0.737865,0.674949,0.042289,0.033116,0.064708,0.079793,0.031907,20.012207,0.188034,0.321367
min,1.00000,23.0,23.00000,74.000000,63.00000,8.000000,13.000000,1.00000,0.000000,0.000000,...,0.000000,0.000000,0.206000,0.284000,0.346000,0.630000,0.294000,64.700000,0.000000,-0.100000
25%,3.25000,23.0,24.00000,80.250000,67.00000,9.250000,18.250000,2.00000,0.000000,0.250000,...,0.250000,0.000000,0.256250,0.344500,0.352750,0.705500,0.331250,88.425000,0.162500,0.067500
50%,5.50000,23.0,24.00000,82.000000,77.00000,11.000000,19.500000,4.00000,0.500000,1.000000,...,1.000000,1.000000,0.279000,0.354000,0.368000,0.736500,0.348000,101.200000,0.215000,0.215000
75%,7.75000,23.0,25.00000,89.500000,78.00000,13.500000,24.750000,5.75000,1.000000,2.000000,...,1.000000,1.000000,0.307250,0.364500,0.419000,0.809000,0.378250,113.375000,0.410000,0.540000
max,10.00000,23.0,31.00000,94.000000,82.00000,20.000000,27.000000,7.00000,1.000000,3.000000,...,2.000000,2.000000,0.342000,0.409000,0.513000,0.876000,0.389000,133.700000,0.590000,0.900000


### 팀명 라벨 인코딩

In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(baseball_data['T_ID']) # 각 카테고리별로 라벨을 만들어주고 
baseball_data.loc[:,'T_ID'] = le.transform(baseball_data['T_ID']) # 생성한 라벨로 문자열 형태의 T_ID를 대체
# loc는 특정 행이나 열을 조회하는 기능을 하는데, 먼저 열에 대해 fit을 사용하고 transform을 사용해 구조를 변형시켜준다...?
# baseball_data.loc[:,'VS_T_ID'] = le.transform(baseball_data['VS_T_ID'])
baseball_data.head()

,순,T_ID,팀,G,AB,PA,RUN,안타,2타,3타,...,희타,희비,OOO,OBP,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1,3,23,25,81,76,14,26,5,1,...,1,1,0.342,0.363,0.513,0.876,0.389,133.7,0.59,0.21
1,2,2,23,30,94,81,16,27,6,0,...,1,1,0.333,0.409,0.407,0.816,0.386,118.2,0.53,0.24
2,3,1,23,31,91,82,20,25,6,1,...,0,2,0.305,0.352,0.512,0.864,0.383,115.5,0.45,0.18
3,4,5,23,24,83,70,12,18,2,0,...,2,1,0.257,0.346,0.371,0.717,0.331,97.7,0.29,0.64
4,5,8,23,24,85,78,10,24,7,1,...,0,0,0.308,0.365,0.423,0.788,0.364,107.0,0.23,0.90


- train_x, train_y : 훈련 데이터
- test_x, test_y : 테스트 데이터

In [10]:
train_data = baseball_data[['RUN','OBP', 'AB', 'PA', 'OOO','T_ID']]
# train_target = baseball_data['win']

In [11]:
import pickle
test_model = pickle.load(open('/Users/jun/Git/407_Project/model/random_forest2.pkl','rb'))
print(test_model.predict(train_data))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [12]:
train_data

,RUN,OBP,AB,PA,OOO,T_ID
0,14,0.363,81,76,0.342,3
1,16,0.409,94,81,0.333,2
2,20,0.352,91,82,0.305,1
3,12,0.346,83,70,0.257,5
4,10,0.365,85,78,0.308,8
5,8,0.356,74,63,0.270,7
6,10,0.392,80,66,0.288,0
7,12,0.344,92,78,0.244,6
8,9,0.342,77,63,0.206,4
9,8,0.284,81,78,0.256,9
